In [2]:
!pip install -U selenium
!apt-get update #atualiza a lista das versões dos pacotes disponíveis
!apt install chromium-chromedriver

     |████████████████████████████████| 958 kB 5.1 MB/s 
     |████████████████████████████████| 138 kB 56.6 MB/s 
     |████████████████████████████████| 356 kB 40.3 MB/s 
     |████████████████████████████████| 55 kB 2.7 MB/s 
     |████████████████████████████████| 3.6 MB 35.9 MB/s 
     |████████████████████████████████| 54 kB 2.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.7 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Hit:14 

In [3]:
import time
import requests as rq
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import json

options = webdriver.ChromeOptions()
options.add_argument('--headless') # não renderiza parte gráfica
options.add_argument('--no-sandbox') #obrigatório, caso contrário gera erro, pois está abrindo como root e exige --no-sandbox

#Veja mais opções aqui:
#https://peter.sh/experiments/chromium-command-line-switches/

driver = webdriver.Chrome(options = options)


In [4]:
url = 'https://store.steampowered.com/specials/?l=portuguese'

driver.get(url)
driver.implicitly_wait(10)

In [5]:
#Função criada para evitar pegar valores nulos e/ou espaço em branco.
def toString(varItem):
    varLista=[]
    for i in varItem:
        if i.text == '':
            pass
        else:
            varLista.append(i.text)
    return varLista

In [6]:
#Pegando o nome dos jogos
jogos_nome = driver.find_elements(By.CLASS_NAME, 'tab_item_name')
nm_jogos = toString(jogos_nome)

In [7]:
#Pegando as classes dos jogos
jogos_details = driver.find_elements(By.CLASS_NAME, 'tab_item_details')
categoria_jogos = toString(jogos_details)

In [8]:
 #Pegando o desconto
 jogos_desconto = driver.find_elements(By.XPATH, '//div[@id="NewReleasesTable"]//div[@class="discount_pct"]')
 desconto_jogos = toString(jogos_desconto)

['-25%', '-10%', '-10%', '-35%', '-20%', '-20%', '-30%', '-10%', '-10%', '-50%', '-30%', '-60%', '-50%', '-35%', '-35%']


In [9]:
#Pegando o valor original
jogos_valOrig = driver.find_elements(By.XPATH, '//div[@id="NewReleasesTable"]//div[@class="discount_original_price"]')
valOrig_jogos = toString(jogos_valOrig)

In [10]:
jogos_valDesc = driver.find_elements(By.XPATH, '//div[@id="NewReleasesTable"]//div[@class="discount_final_price"]')
valDesc_jogos = toString(jogos_valDesc)

In [11]:
df = pd.DataFrame() 

In [15]:
df['nomes'] = nm_jogos
df['categorias'] = categoria_jogos
df['perc_desc'] = desconto_jogos
df['val_original'] = valOrig_jogos
df['val_desconto'] = valDesc_jogos
print(df.to_string())

df.to_csv('jogos_desconto.csv',encoding='latin1',index=False)

                               nomes                                                                          categorias perc_desc val_original val_desconto
0                               GTFO                        Co-op, Co-op Online, Tiros em Primeira Pessoa, Jogo de Tiros      -25%       $39.99       $29.99
1     Ultimate Admiral: Dreadnoughts                                  Combate Naval, Naval, Simulador, Acesso Antecipado      -10%       $34.99       $31.49
2   After the Fall® - Launch Edition                  Só em R.V.\nAção, Aventura, Violência Explícita, Realidade Virtual      -10%       $39.99       $35.99
3   Marvel's Guardians of the Galaxy                                 Ação, Exploração, Ação/Aventura, História Excelente      -35%       $59.99       $38.99
4                   Escape Simulator                                 Quebra-Cabeças, Co-op, Escape Room, Primeira Pessoa      -20%       $14.99       $11.99
5                        Inscryption  Card Battler, Jogo d